## 9 包
我们将以“如何把代码组织成包结构”这一主题结束本课程。同时，也将对第三方包的安装、如何将自己的代码提供给其他人使用进行讨论。

在 Python 开发中，打包（packaging）是一个演进的，复杂的主题。本节主要关注一些通用的代码组织原则，而不是特定的工具。无论以后使用那种工具来分发代码或者管理依赖，这些通用的代码组织原则都是有用的。

### 01  包
如果编写一个较大的程序，我们并不真的想在顶层将其组织为一个个独立文件的大型集合。本节对包（package）进行介绍。
#### **模块**
任何一个 Python 源文件称为一个模块（module）。
```
# foo.py
def grok(a):
    ...
def spam(b):
    ...
```
一条 `import` 语句加载并执行 一个模块。
```
# program.py
import foo

a = foo.grok(2)
b = foo.spam('Hello')
...
```
#### **包 vs 模块**
对于较大的代码集合，通常将模块组织到包中。
```
# From this
pcost.py
report.py
fileparse.py

# To this
porty/
    __init__.py
    pcost.py
    report.py
    fileparse.py
```
首先，选择一个名字并用该名字创建顶级目录。如上述的 `porty` （显然，第一步最重要的是选择名字）。

接着，添加 `__init__.py` 文件到该目录中。`__init__.py` 文件可以是一个空文件。

最后，把源文件放到该目录中。
#### **使用包**
包用作导入的命名空间。

这意味着现在有了多级导入。
```
import porty.report
port = porty.report.read_portfolio('port.csv')
```
导入语句还有其它变体：
```
from porty import report
port = report.read_portfolio('portfolio.csv')

from porty.report import read_portfolio
port = read_portfolio('portfolio.csv')
```
#### **两个问题**
这种方法存在两个主要的问题：
* 同一包内不同文件之间的导入无效。
* 包中的主脚本无效。
因此，基本上一切导入都是无效的，但是，除此之外，程序还是可以工作的。
#### **问题：导入**
现在，在导入的时候，同一包内的不同文件之间的导入必须包含包名。请记住这个结构：

点击porty，则可以跳转到该目录，则其下python文件都在里面。
```
porty/
    __init__.py
    pcost.py
    report.py
    fileparse.py
run.py
```
根据上述规则（同一包内的不同文件之间的导入必须包含包名）修改后的导入示例：
```
# report.py
from porty import fileparse

def read_portfolio(filename):
    return fileparse.parse_csv(...)
```
所有的导入都是绝对的，而不是相对的。
```
# report.py
import fileparse    # BREAKS. fileparse not found

...
```
#### **相对导入**
除了使用包名直接导入，还可以使用使用 . 引用当前的包。
```
# report.py
from . import fileparse

def read_portfolio(filename):
    return fileparse.parse_csv(...)
```
语法:
```
from . import modname
```
使用上述语法使得重命名包变得容易。

#### **问题：主脚本**
将包内的子模块作为主脚本运行会导致程序中断：
```
bash $ python porty/pcost.py # BREAKS
...
```
原因：你正在运行单个脚本，而 Python 不知道包的其余部分（`sys.path `是错误的）。

所有的导入都会中断。要想解决这个问题，需要以不同的方式运行程序，可以使用 -m 选项。
```
bash $ python -m porty.pcost # WORKS
...
```
#### **__init__.py 文件**
该文件的主要目的是将模块组织在一起。

例如：
```
# porty/__init__.py
from .pcost import portfolio_cost
from .report import portfolio_report
```
这使得导入的时候名字出现在顶层。
```
from porty import portfolio_cost
portfolio_cost('portfolio.csv')
```
而不是使用多级导入：
```
from porty import pcost
pcost.portfolio_cost('portfolio.csv')
```
#### **脚本的另一种解决方案**
如前所述，需要使用 `-m package.module` 运行包内的脚本。
```
bash % python3 -m porty.pcost portfolio.csv
```
还有另一种选择：编写一个新的顶级脚本。
```
#!/usr/bin/env python3
# pcost.py
import porty.pcost
import sys
porty.pcost.main(sys.argv)
```
脚本位于包外面。目录结构如下：
```
pcost.py       # top-level-script
porty/         # package directory
    __init__.py
    pcost.py
    ...
```
#### **应用结构**
代码组织和文件结构是应用程序可维护性的关键。

对于 Python 而言，没有“放之四海而皆准”的方法，但是一个适用于多种问题的结构就是这样：
```
porty-app/
  README.txt
  script.py         # SCRIPT
  porty/
    # LIBRARY CODE
    __init__.py
    pcost.py
    report.py
    fileparse.py
```
顶级 porty-app 目录是所有其他内容的容器——这些内容包括文档，顶级脚本，用例等。

同样，顶级脚本（如果有）需要放置在代码包之外（包的上一层）。
```
#!/usr/bin/env python3
# porty-app/script.py
import sys
import porty

porty.report.main(sys.argv)
```

### 练习
此时，我们有了一个包含多个程序的目录：
```
pcost.py          # computes portfolio cost
report.py         # Makes a report
ticker.py         # Produce a real-time stock ticker
```
同时，还有许多具有各种功能的支持模块：
```
stock.py          # Stock class
portfolio.py      # Portfolio class
fileparse.py      # CSV parsing
tableformat.py    # Formatted tables
follow.py         # Follow a log file
typedproperty.py  # Typed class properties
```
在本次练习中，我们将整理这些代码并将它们放入一个通用包中。
#### **练习 9.1：创建一个简单的包**
请创建一个名为 porty 的目录并将上述所有的 Python 文件放入其中。另外，在 `porty` 目录中创建一个空的 `__init__.py` 文件。最后，文件目录看起来像这样：
```
porty/
    __init__.py
    fileparse.py
    follow.py
    pcost.py
    portfolio.py
    report.py
    stock.py
    tableformat.py
    ticker.py
    typedproperty.py
```
请将 `porty` 目录中的 `__pycache__` 目录移除。该目录包含了之前预编译的 Python 模块。我们想重新开始。

尝试导入包中的几个模块：
```
>>> import porty.report
>>> import porty.pcost
>>> import porty.ticker
```
如果这些导入失败，请进入到合适的文件中解决模块导入问题，使其能够包括相对导入。例如，`import fileparse` 语句可以像下面这样进行修改：
```
# report.py
from . import fileparse
...
```
如果有类似于 `from fileparse import parse_csv` 这样的语句，请像下面这样修改代码：
```
# report.py
from .fileparse import parse_csv
...
```

#### **练习 9.2：创建应用目录**
对应用而言，将所有代码放到“包”中通常是不够的。有时，支持文件，文档，脚本等文件需要放到 porty/ 目录之外。

请创建一个名为 `porty-app` 的新目录。然后将我们在练习 9.1 中创建的 `porty` 目录移动到 `porty-app` 目录中。接着，复制测试文件 `Data/portfolio.csv` 和 `Data/prices.csv` 到 porty-app 目录。另外，在 `porty-app` 目录下创建一个 `README.txt` 文件，该文件包含一些有关自己的信息。现在，代码的组织结构像下面这样：
```
porty-app/
    portfolio.csv
    prices.csv
    README.txt
    porty/
        __init__.py
        fileparse.py
        follow.py
        pcost.py
        portfolio.py
        report.py
        stock.py
        tableformat.py
        ticker.py
        typedproperty.py
```
要运行代码，需要确保你现在正在顶级目录 `porty-app/` 下。例如，从终端运行：
```
shell % cd porty-app
shell % python3
>>> import porty.report
>>>
```
尝试将之前的脚本作为主程序运行：
```
shell % cd porty-app
shell % python3 -m porty.report portfolio.csv prices.csv txt
      Name     Shares      Price     Change
---------- ---------- ---------- ----------
        AA        100       9.22     -22.98
       IBM         50     106.28      15.18
       CAT        150      35.46     -47.98
      MSFT        200      20.89     -30.34
        GE         95      13.48     -26.89
      MSFT         50      20.89     -44.21
       IBM        100     106.28      35.84

shell %
```

#### **练习 9.3：顶级脚本**
使用 `python -m` 命令通常有点怪异。可能需要编写一个顶级脚本来处理奇怪的包。请创建一个生成上述报告的脚本 `print-report.py`：
```
#!/usr/bin/env python3
# print-report.py
import sys
from porty.report import main
main(sys.argv)
```
然后把脚本 `print-report.py` 放到顶级目录 `porty-app/ `中。并确保可以在 `porty-app/` 目录下运行它：
```
shell % cd porty-app
shell % python3 print-report.py portfolio.csv prices.csv txt
      Name     Shares      Price     Change
---------- ---------- ---------- ----------
        AA        100       9.22     -22.98
       IBM         50     106.28      15.18
       CAT        150      35.46     -47.98
      MSFT        200      20.89     -30.34
        GE         95      13.48     -26.89
      MSFT         50      20.89     -44.21
       IBM        100     106.28      35.84

shell %
```
最后，代码的组织结构应该下面这样：
```
porty-app/
    portfolio.csv
    prices.csv
    print-report.py
    README.txt
    porty/
        __init__.py
        fileparse.py
        follow.py
        pcost.py
        portfolio.py
        report.py
        stock.py
        tableformat.py
        ticker.py
        typedproperty.py
```

### 02 第三方模块
Python 拥有一个包含各种内置模块的大型库（自带电池(batteries included)）（译注：“自带电池”来自于官方文档的翻译，意为“功能齐全”）。

甚至还有很多第三方模块（可以使用）。请到 Python 包索引（ PyPi ） 查看，或者使用谷歌搜索特定主题。

对于 Python 而言，如何处理第三方依赖关系是一个不断演化的主题。本节仅仅涵盖一些基础知识，帮助你了解它们是如何工作的。
#### **模块搜索路径**
`sys.path` 是一个列表，包含所有` import` 语句要检查的目录。查看 `sys.path` ：
```
>>> import sys
>>> sys.path
... look at the result ...
>>>
```
如果导入的内容不在目录中。那么将会触发 `ImportError` 异常。
#### **标准库模块**
Python 标准库中的模块通常来自于 `/usr/local/lib/python3.6` 之类的位置。可以通过一个简短的测试来确定模块来自于哪里：
```
>>> import re
>>> re
<module 're' from '/usr/local/lib/python3.6/re.py'>
>>>
```
在交互式解释器（REPL）中查看模块的位置是一个很好的调试技巧。交互式解释器将会显示模块所在的位置。
#### **第三方模块**
第三方模块通常位于专门的目录 `site-packages` 中。如果执行与上述相同的步骤，可以看到第三方模块所在的位置：
```
>>> import numpy
>>> numpy
<module 'numpy' from '/usr/local/lib/python3.6/site-packages/numpy/__init__.py'>
>>>
```
同样，如果要弄清楚导入的第三方模块为什么没有像预期那样工作，那么查看模块的位置是一个很好的调试技巧。
#### **安装模块**
安装第三方模块最常用的技术是使用 pip。示例：
```
bash % python3 -m pip install packagename
```
此命令会下载包并将包安装到 `site-packages` 目录中。
#### **问题**
* 你可能正在使用你不能直接控制的 Python。
  - 企业认可的 Python
  - 操作系统自带的 Python
* 你可能没有权限在计算机中安装全局包
* 可能还有其它依赖
#### **虚拟环境**
对于包安装问题，一个常见的解决方案是为自己创建所谓的“虚拟环境”。自然，创建虚拟环境的方式不止这一种——实际上，有好几种相互竞争的工具和技术。如果你正在使用标准方式安装的 Python，可以尝试输入以下代码创建虚拟环境：
```
bash % python -m venv mypython
bash %
```
稍等片刻后，你将拥有一个新目录 `mypython`，该目录安装了你自己的 Python。在 `mypython` 目录中，可以找到 `bin/` 目录（Unix 系统）或者 `Scripts/` 目录（Windows 系统）。运行 `bin/` 目录下的 `activate` 脚本会“激活“ 对应的 `Python`，使其成为 `shell` 的默认 `python` 命令。示例：
```
bash % source mypython/bin/activate
(mypython) bash %
```
现在，你可以从这里开始安装自己的 Python 包了。示例：
```
(mypython) bash % python -m pip install pandas
...
```
通常，为了试验和试用不同的包，可以使用虚拟环境。另一方面，如果你正在创建一个应用并且它有特定的包依赖关系，那么问题会稍有不同。
#### **处理应用中的第三方依赖**
如果你已经编写了一个应用，并且该应用具有特定的第三方依赖，那么创建并保存一个包括你自己的代码及其依赖的环境将会是一个挑战。可悲的是，这一直是一个非常混乱和频繁变化的领域。即使到现在，它仍在继续演变。

我不想向你提供必定会过时的信息，建议查阅 Python 打包用户指南 。

### 练习
#### **练习 9.4：创建虚拟环境**
看看你是否可以像上面一样创建一个虚拟环境并在虚拟环境中安装 pandas。

### 03 将代码提供给其他人（使用）
在某些时候，你可能想要将自己的代码提供给其他人——可能只是同事（使用）。本节给出执行此操作的最基本技术。更多详细信息，请参考 Python 打包用户指南。
#### **创建 setup.py 文件**
请添加一个 `setup.py` 到项目目录的顶层。
```
# setup.py
import setuptools

setuptools.setup(
    name="porty",
    version="0.0.1",
    author="Your Name",
    author_email="you@example.com",
    description="Practical Python Code",
    packages=setuptools.find_packages(),
)
```
#### **创建 MANIFEST.in 文件**

如果有其它文件与你的项目相关联，请使用一个 MANIFEST.in 文件指定这些关联的文件。示例：

# MANIFEST.in
include *.csv

请将 MANIFEST.in 文件放到 setup.py 所在的目录。
#### **创建源码发行版**
要创建源码发行版，请使用 setup.py 文件。示例：
```
bash % python setup.py sdist
```
这将在 `dist/` 目录中创建 `.tar.gz` 或者 `.zip` 文件。该文件就是你要提供给其他人使用的文件。
#### **安装你的代码**
其他人可以使用 `pip` 像安装其它软件包一样安装你的 Python 代码。他们仅仅需要提供在之前的步骤中创建的文件即可。示例：
```
bash % python -m pip install porty-0.0.1.tar.gz
```
#### **说明**
上面的步骤描述了创建 Python 代码包的最基本的知识，您可以将这些代码包提供给其他人使用。实际上，根据第三方的依赖关系，它可能要复杂得多，无论你的应用是否包含外部代码(例如 C 或 C++)。这已经超出了本课程的范围，我们只作初步了解。
### 练习
#### **练习 9.5：创建软件包**
使用练习 9.3 中创建的 `porty-app/` 代码，看看是否可以重新创建此前描述的步骤。具体来说，添加一个 `setup.py` 文件和一个 `MANIFEST.in` 文件到顶级目录中。然后通过运行 `python setup.py sdist` 创建源码发行版。

最后，看看是否可以在 Python 虚拟环境中安装你的软件包。